<a href="https://colab.research.google.com/github/HosseinZaredar/Transformer-from-Scratch/blob/main/transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import torch as T
import torch.nn as nn
from torchtext.legacy import data, datasets
import torch.optim as optim
import time

### Embedding Layer (Word + Positional)

Instead of using a sinusoidal positional encoding (as in the original paper), a learnable positional embedding is used.

In [ ]:
class Embedding(nn.Module):
  def __init__(self, vocab_size, max_length, embed_dim, dropout=0.1):
    super(Embedding, self).__init__()
    self.word_embed = nn.Embedding(vocab_size, embed_dim)
    self.pos_embed = nn.Embedding(max_length, embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    batch_size, seq_length = x.shape
    device = T.device('cuda' if T.cuda.is_available() else 'cpu')
    positions = T.arange(0, seq_length).expand(
        batch_size, seq_length).to(device)
    embedding = self.word_embed(x) + self.pos_embed(positions)
    return self.dropout(embedding)

### Multi-Head Self-Attention

<center>
<img src="https://raw.githubusercontent.com/HosseinZaredar/Transformer-from-Scratch/main/SelfAttention.png" width="600" align="center"/>
</center>


In [ ]:
class MHSelfAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super(MHSelfAttention, self).__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.head_dim = embed_dim // num_heads

    assert (self.num_heads*self.head_dim == self.embed_dim), \
    'embed size must be divisible by number of heads'

    self.w_queries = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
    self.w_keys = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
    self.w_values = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
    
    self.fc_out = nn.Linear(self.head_dim*self.num_heads , self.embed_dim)

  def forward(self, x):

    # shape of x = [batch_size, sentence_length, embedding_dim]
    batch_size = x.shape[0]
    sentence_len = x.shape[1]

    queries = self.w_queries(x).reshape(
        batch_size, sentence_len, self.num_heads, self.head_dim).permute(
            0, 2, 1, 3)
    
    keys = self.w_keys(x).reshape(
        batch_size, sentence_len, self.num_heads, self.head_dim).permute(
            0, 2, 3, 1)
    
    
    values = self.w_values(x).reshape(
        batch_size, sentence_len, self.num_heads, self.head_dim).permute(
            0, 2, 1, 3)
    
    attention_scores = T.einsum('bijk,bikl->bijl', queries, keys)
    attention_dist = T.softmax(attention_scores /
                               (self.embed_dim ** (1/2)), dim=-1)
    attention_out = T.einsum('bijk,bikl->bijl', attention_dist, values)
    concatenated_out = attention_out.permute(0, 2, 1, 3).reshape(
        batch_size, sentence_len, self.embed_dim)
    
    return concatenated_out

### Transformer Encoder

<center>
<img src="https://raw.githubusercontent.com/HosseinZaredar/Transformer-from-Scratch/main/Encoder.png" width="200" align="center"/>
</center>

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, embed_dim, num_heads, forward_expansion, dropout=0.1):
    super(TransformerEncoder, self).__init__()

    self.attention = MHSelfAttention(embed_dim, num_heads)
    self.norm1 = nn.LayerNorm(embed_dim)
    self.norm2 = nn.LayerNorm(embed_dim)

    self.feed_forward = nn.Sequential(
        nn.Linear(embed_dim, forward_expansion*embed_dim),
        nn.ReLU(),
        nn.Linear(forward_expansion*embed_dim, embed_dim)
    )
    
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    attention_out = self.dropout(self.attention(x))
    x = self.norm1(x + attention_out)  
    forward_out = self.dropout(self.feed_forward(x))
    out = self.norm2(x + forward_out)

    return out

### End-to-End Classifier

1. An embedding layer
2. A single transformer encoder layer
3. A fully-connected network as a linear classifier

In [ ]:
class Classifier(nn.Module):
  def __init__(self, vocab_size, max_length, embed_dim,
               num_heads, forward_expansion):
      super(Classifier, self).__init__()

      self.embedder = Embedding(vocab_size, max_length, embed_dim)
      self.encoder = TransformerEncoder(embed_dim, num_heads, forward_expansion)
      self.fc = nn.Linear(embed_dim, 1)

  def forward(self, x):
    embedding = self.embedder(x)
    encoding = self.encoder(embedding)
    compact_encoding = encoding.max(dim=1)[0]
    out = self.fc(compact_encoding)
    return out

### Load and Preprocess IMDb Dataset

In [ ]:
TEXT = data.Field(
    tokenize = 'spacy', tokenizer_language = 'en_core_web_sm',batch_first=True)
LABEL = data.LabelField(dtype = T.float)

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
train_data, valid_data = train_data.split()

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 4
device = T.device('cuda' if T.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### Initializing The Model

In [ ]:
EMBED_DIM = 200
NUM_HEADS = 8
FORWARD_EXPANSION = 3
MAX_LENGTH = 512
VOCAB_SIZE = len(TEXT.vocab)

classifier = Classifier(
    VOCAB_SIZE, MAX_LENGTH, EMBED_DIM, NUM_HEADS, FORWARD_EXPANSION)
device = T.device('cuda' if T.cuda.is_available() else 'cpu')
classifier.to(device)

Classifier(
  (embedder): Embedding(
    (word_embed): Embedding(25002, 200)
    (pos_embed): Embedding(512, 200)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (attention): MHSelfAttention(
      (w_queries): Linear(in_features=200, out_features=200, bias=False)
      (w_keys): Linear(in_features=200, out_features=200, bias=False)
      (w_values): Linear(in_features=200, out_features=200, bias=False)
      (fc_out): Linear(in_features=200, out_features=200, bias=True)
    )
    (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (feed_forward): Sequential(
      (0): Linear(in_features=200, out_features=600, bias=True)
      (1): ReLU()
      (2): Linear(in_features=600, out_features=200, bias=True)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (fc): Linear(in_features=200, out_features=1, bias=True)
)

### Training

In [ ]:
optimizer = optim.SGD(classifier.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
device = T.device('cuda' if T.cuda.is_available() else 'cpu')
criterion.to(device);

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = T.round(T.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        input = batch.text
        if input.shape[1] > MAX_LENGTH:
          input = input[:, :MAX_LENGTH]
                
        predictions = model(input).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with T.no_grad():
        for batch in iterator:

            input = batch.text
            if input.shape[1] > MAX_LENGTH:
              input = input[:, :MAX_LENGTH]
                    
            predictions = model(input).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(
        classifier, train_iterator,optimizer, criterion)
    valid_loss, valid_acc = evaluate(classifier, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        T.save(classifier.state_dict(), 'sent-classifier.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 2s
	Train Loss: 0.730 | Train Acc: 49.83%
	 Val. Loss: 0.702 |  Val. Acc: 50.51%
Epoch: 02 | Epoch Time: 1m 1s
	Train Loss: 0.719 | Train Acc: 51.80%
	 Val. Loss: 0.688 |  Val. Acc: 53.43%
Epoch: 03 | Epoch Time: 1m 1s
	Train Loss: 0.704 | Train Acc: 53.22%
	 Val. Loss: 0.689 |  Val. Acc: 53.61%
Epoch: 04 | Epoch Time: 1m 1s
	Train Loss: 0.695 | Train Acc: 55.47%
	 Val. Loss: 0.683 |  Val. Acc: 55.73%
Epoch: 05 | Epoch Time: 1m 1s
	Train Loss: 0.684 | Train Acc: 57.75%
	 Val. Loss: 0.684 |  Val. Acc: 56.09%
Epoch: 06 | Epoch Time: 1m 1s
	Train Loss: 0.670 | Train Acc: 59.65%
	 Val. Loss: 0.641 |  Val. Acc: 62.57%
Epoch: 07 | Epoch Time: 1m 1s
	Train Loss: 0.648 | Train Acc: 62.01%
	 Val. Loss: 0.618 |  Val. Acc: 65.43%
Epoch: 08 | Epoch Time: 1m 1s
	Train Loss: 0.627 | Train Acc: 64.58%
	 Val. Loss: 0.595 |  Val. Acc: 67.84%
Epoch: 09 | Epoch Time: 1m 1s
	Train Loss: 0.602 | Train Acc: 67.27%
	 Val. Loss: 0.586 |  Val. Acc: 68.76%
Epoch: 10 | Epoch Time: 1m 1

### Evaluation

In [ ]:
classifier.load_state_dict(T.load('sent-classifier.pt'))

<All keys matched successfully>

In [ ]:
test_loss, test_acc = evaluate(classifier, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.578 |  Test Acc: 69.58%
